## Regression of the energy response distribution
Here we will try not just to evaluate the best calibrated energy, but to get the distribution of the response. Basically we will get the distribution of $E_\text{raw}/E_\text{true}$ (and not only its best value).

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
import scipy
tfd = tfp.distributions

## Import data
Use the same inputs as in the previous simpler energy regression

In [ ]:
df_train = pd.read_csv('http://rgw.fisica.unimi.it/TutorialML-AtlasItalia2022/train_electron_Et0-10_eta1.0-1.2_Eaccordion.csv?AWSAccessKeyId=M06HBTUGIKXVXYH1RES6&Signature=U%2BMJxVi5El1wxtCz%2B45VqLmUuok%3D&Expires=1828739034')
df_test = pd.read_csv('http://rgw.fisica.unimi.it/TutorialML-AtlasItalia2022/test_electron_Et0-10_eta1.0-1.2_Eaccordion.csv?AWSAccessKeyId=M06HBTUGIKXVXYH1RES6&Signature=YKG4lzc%2FI0%2BcJZRnQG350DnVVK4%3D&Expires=1828739085')

#df_train = pd.read_csv('train_electron_Et0-10_eta1.0-1.2_Eaccordion.csv')
#df_test = pd.read_csv('test_electron_Et0-10_eta1.0-1.2_Eaccordion.csv')

df_train['el_rawcl_Es1Over2'] = df_train['el_rawcl_Es1'] / df_train['el_rawcl_Es2']
df_test['el_rawcl_Es1Over2'] = df_test['el_rawcl_Es1'] / df_test['el_rawcl_Es2']

In [ ]:
columns_X = ['el_rawcl_E', 'el_rawcl_Es1Over2', 'el_f0', 'el_cl_aeta', ]  # input variables
column_y = 'el_erawOverEtrue'

normalizer = tf.keras.layers.Normalization()
normalizer.adapt(np.array(df_train[columns_X]))

## Model

## Pdf to be used
We have to choose the type of the pdf to use. To be generic, use a mixture of normal distributions, e.g.

$$
  \sum_{i=1}^3 \alpha_i \times N[\mu_i, \sigma^2_i]
$$

The algorithm will learn the parameters (9 in this case) as a function of the input variables

In [ ]:
event_shape = [1]  # the energy if 1D scalar quantity -> 1D pdf
num_components = 3 # number of component of the pdf mixture (3 gaussian)
params_size = tfp.layers.MixtureNormal.params_size(num_components, event_shape)
params_size  # number of parameter of the final pdf (3 fractions, 3 means, 3 variances)

In [ ]:
pdf_template = tfp.layers.MixtureNormal(num_components, event_shape)
#tfp.layers.MixtureSameFamily(num_components, tfp.layers.IndependentNormal(event_shape))

model = tf.keras.Sequential([
    tf.keras.Input(shape=len(columns_X)),
    normalizer,    
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(params_size, activation=None),
    pdf_template,
])

tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.summary()

## Define the loss
This time we will define manually the loss. We will use the negative log-likelihood

In [ ]:
negloglik = lambda y, p_y: -p_y.log_prob(y)
# note: it would be better to add some regularization terms, constraining the parameters to don't be degenerate, ...

## Train

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss=negloglik)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(df_train[columns_X].values, df_train[column_y].values,
                    epochs=50, batch_size=1024, validation_split=0.2, callbacks=[callback])

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()

## Apply the model to the test sample
Note that the output is not a value, but a distribution

In [ ]:
yhat = model(df_test[columns_X].values)  # note we are not using predict
df_results_tf = df_test.copy()

In [ ]:
yhat

## Plot the parameters of the estimated distributions

In [ ]:
alphas = yhat.mixture_distribution.probs_parameter().numpy()
means = np.squeeze(yhat.components_distribution.mean().numpy())
stds = np.sqrt(np.squeeze(yhat.components_distribution.variance().numpy()))

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
for ax, values, name in zip(axs.flat, (alphas, means, stds), ('alphas', 'means', 'stds')):
    m, M = np.quantile(values, (0.01, 0.99))
    for icomponent in range(num_components):
        ax.hist(values[:, icomponent], bins=np.linspace(m, M, 100),
                histtype='stepfilled',
                label=f'component {icomponent}')
    ax.legend(loc=0)
    ax.set_title(name, fontsize=15)
plt.show()

## Plot the mean and std of the estimated response
For each event we can estimated the resolution!

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

df_test['NN_mean'] = yhat.mean().numpy().flatten()
df_test['NN_std'] = np.sqrt(yhat.variance().numpy().flatten())


axs[0].hist(df_test['NN_mean'], bins=100, density=True)
axs[0].set_xlabel('mean of the estimated response', fontsize=15)

axs[1].hist(df_test['NN_std'], bins=100)
axs[1].set_xlabel('std of the estimated response', fontsize=15)
plt.show()

## Respose of the first events
As example plot the response evaluated by the model for the fist events in the test sample.

In [ ]:
xspace = np.linspace(0.2, 1.3, 100)

for idx in range(3):
    ys = scipy.stats.norm(means[idx], stds[idx]).pdf(np.tile(xspace, (3, 1)).T)
    ys = ys * alphas[idx]
    y = ys.sum(axis=1)
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.fill_between(xspace, y, color='0.7', label='NN')
    ax.plot(xspace, ys, label=[f'NN component {icomp}' for icomp in range(num_components)])
    ax.legend(loc=0)
    ax.set_title('event %d, rawE = %.0f GeV, |eta| = %.1f' % (idx, df_test.loc[idx, "el_rawcl_E"] / 1E3, df_test.loc[idx, "el_cl_aeta"]))
    ax.set_xlabel('response = Eraw / Etrue', fontsize=12)

The orange component is very small, probably it would be good to introduce a regularization to avoid it, or maybe it is not needed. It is also very large: it is describing outliers

The green and blue have very different mean, which means that the model is able to describe the asymmetri of the response.

## Check the model response with the test sample
Evalue if the model is able to reproduce the distribution of the response from the test dataset

In [ ]:
xspace = np.linspace(0.1, 1.4, 100)

y = model(df_test[columns_X].values).tensor_distribution.prob(xspace.reshape(-1, 1, 1))
ysum = np.sum(y, axis=1)

fig, ax = plt.subplots(figsize=(8, 8))
ax.plot(xspace, ysum / len(df_test), label='NN model')
ax.hist(df_test[column_y], bins=50, density=True, label='test sample')
ax.legend(loc=0, fontsize=14)
ax.set_xlabel('response = Eraw / Etrue', fontsize=14)
plt.show()

In [ ]:
ax.set_yscale('log')  # look at the tails, quite well modelled
fig

## Resolution estimation
We are able to estimate the resolution event by event! Check it differentially as a function of the input variables

Note how smooth is the NN

In [ ]:
def get_profile(df, xvar, energy_var, xedges, estimators=('median', 'sem')):
    df_agg = df_test.groupby(np.digitize(df_test[xvar], xedges)).apply(
        # approximate the error of the median as the standard error on the mean (sem)
        lambda df: (df[energy_var] / df['el_truth_E']).apply(estimators))
    df_agg = df_agg.reindex(range(1, len(xedges)))
    xmidpoints = 0.5 * (xedges[1:] + xedges[:-1])
    df_agg.index = xmidpoints
    return df_agg

for xvar in columns_X:
    xedges = df_test[xvar].quantile(np.linspace(0, 1, 20)).values

    fig, ax = plt.subplots(figsize=(8, 7))

    df_agg1 = get_profile(df_test, xvar, 'el_rawcl_E', xedges, ('std',) )
    ax.plot(df_agg1.index, df_agg1.values, '.-', label='test sample')

    xedges = df_test[xvar].quantile(np.linspace(0, 1, 50)).values
    df_agg2 = df_test.groupby(np.digitize(df_test[xvar], xedges))['NN_std'].mean()
    df_agg2 = df_agg2.reindex(range(1, len(xedges)))
    ax.plot(0.5 * (xedges[1:] + xedges[:-1]), df_agg2.values, '.-', label='NN')
    ax.legend(fontsize=14)
    ax.set_xlabel(xvar, fontsize=14)
    ax.set_ylabel('resolution', fontsize=14)
    ax.set_ylim(0.05, 0.13)
    
    ax2 = ax.twinx()
    ax2.hist(df_test[xvar], 50, color='0.7')
    ax2.set_ylim(0, ax2.get_ylim()[1] * 1.5)
    
    ax.set_zorder(ax2.get_zorder() + 1)
    ax.set_frame_on(False)
    plt.show()

## Estimate the best energy
For each event we can estimate the best energy

   * for comparison, the raw energy scaled by the  median response (evaluated on the test dataset)
   * using as correction the mean of the estimated distribution
   * using as correction the mean of the largest component of the estimated distribution (as an approximation of the mode)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))
bins = np.linspace(0.5, 1.4, 100)

ax.hist(df_test['el_rawcl_E'] / df_test['el_erawOverEtrue'].median() / df_test['el_truth_E'], label='raw median scaled', bins=bins, density=True, alpha=0.6)

correction_mean = yhat.mean().numpy().flatten()
ax.hist(df_test['el_rawcl_E'] / correction_mean / df_test['el_truth_E'], label='NN from mean', bins=bins, density=True, histtype='step', lw=2)

correction_dominant_mean = yhat.components_distribution.mean().numpy()[:, 1, 0]
ax.hist(df_test['el_rawcl_E'] / correction_dominant_mean / df_test['el_truth_E'], label='NN from dominant mean', bins=bins, density=True, histtype='step', lw=2)

ax.axvline(1, ls='--', color='0.3')
ax.legend(loc=2, fontsize=13)
ax.set_xlabel('E / Etrue', fontsize=14)